<a href="https://colab.research.google.com/github/s34836/BIG_DATA/blob/main/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: add to the array pair of word and number

def add_word_number_pair(arr, word, number):
  arr.append((word, number))
  return arr



In [8]:
#!/usr/bin/env python
import sys
import re
mapper = []
# Replace sys.stdin with file input
with open('short_story.txt', 'r') as f:
    for line in f:
        line = re.sub(r'\W+', ' ', line.strip())
        words = line.split()
        for word in words:
            mapper.append((word, 1))



In [7]:
#!/usr/bin/env python

import sys

current_word = None
current_count = 0
word = None

for word, number in mapper:
      line = line.strip()
      word, count = line.split('\t',1)

      try:
           count = int(count)
      except ValueError:
           continue

      if current_word == word:
           current_count += count
      else:
           if current_word:
                print('{}\t{}'.format(current_word,current_count))
           current_word = word
           current_count = count
 if current_word == word:
      print('{}\t{}'.format(current_word,current_count))


Death	1
was	21
accompanied	1
by	12
the	112
patter	1
of	50
nurses	2
rubber	1
soled	1
shoes	1
marching	1
briskly	1
along	1
linoleum	1
floors	1
The	10
smell	1
antiseptic	1
solution	1
A	2
gurney	1
wheel	1
in	41
need	1
oil	1
squeaking	1
rhythmically	1
as	20
another	3
patient	1
rushed	1
past	3
Starchy	1
white	3
sheets	2
and	62
sharp	1
angular	1
shapes	1
headdresses	1
My	6
body	6
revolted	1
against	4
crisp	1
order	1
violent	2
expulsions	1
browns	1
greens	1
yellows	1
bleached	1
hospital	3
whites	1
Veins	1
stood	3
out	4
on	16
my	35
small	5
pale	1
neck	2
spittle	1
flew	1
from	14
mouth	4
I	68
screamed	2
cool	4
bed	2
tucked	1
with	22
neat	2
stern	1
corners	1
were	9
twisted	3
into	8
angry	1
damp	3
knots	1
writhing	1
am	7
ordinarily	2
a	41
demure	1
person	1
so	4
you	2
can	5
imagine	1
cringing	1
horror	1
at	12
this	6
loud	2
messy	1
departure	1
shudder	1
when	7
mind	2
invariably	1
drifts	2
to	40
remember	4
it	7
happily	1
brought	1
back	2
present	2
distant	2
iron	2
clang	1
cemetery	3
gate	5
There	4
s	9

In [10]:
!pip install multiprocess

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.0 MB/s eta 0:00:00


In [11]:
from collections import defaultdict
from multiprocess import Pool, cpu_count
from typing import Callable, List, Tuple, Any

class MapReduce:
    def __init__(self, mapper: Callable[[Any], List[Tuple[Any, Any]]],
                 reducer: Callable[[Any, List[Any]], Any]):
        """
        Initialize with custom mapper and reducer functions.

        :param mapper: A function that maps input to key-value pairs.
        :param reducer: A function that reduces key-value pairs to output.
        """
        self.mapper = mapper
        self.reducer = reducer

    def _map_worker(self, data_chunk):
        """Process data chunk through the mapper."""
        return self.mapper(data_chunk)

    def _reduce_worker(self, kv_pairs):
        """Process key-value pairs through the reducer."""
        key, values = kv_pairs
        return self.reducer(key, values)

    def execute(self, data: List[Any], num_workers: int = None) -> List[Tuple[Any, Any]]:
        """
        Execute the MapReduce operation.

        :param data: Input data to be processed.
        :param num_workers: Number of workers for parallel processing (defaults to number of CPU cores).
        :return: The final reduced result as a list of key-value pairs.
        """
        if num_workers is None:
            num_workers = cpu_count()

        # Map Phase: Apply mapper function to the input data in parallel.
        with Pool(num_workers) as pool:
            mapped = pool.map(self._map_worker, data)

        # Combine all mapped results into a list of key-value pairs.
        kv_store = defaultdict(list)
        for sublist in mapped:
            for key, value in sublist:
                kv_store[key].append(value)

        # Reduce Phase: Apply reducer function to each key's values.
        with Pool(num_workers) as pool:
            reduced = pool.map(self._reduce_worker, kv_store.items())

        return reduced

# Example: Word Count MapReduce

def word_count_mapper(document: str) -> List[Tuple[str, int]]:
    """
    Mapper function that splits a document into words and emits key-value pairs of (word, 1).
    """
    words = document.split()
    return [(word, 1) for word in words]

def word_count_reducer(word: str, counts: List[int]) -> Tuple[str, int]:
    """
    Reducer function that sums the occurrences of each word.
    """
    return word, sum(counts)

if __name__ == "__main__":
    # Sample Data: List of documents (strings)
    documents = [
        "hello world",
        "hello from the other side",
        "hello there",
        "world of programming"
    ]

    # Initialize MapReduce with the custom mapper and reducer.
    map_reduce = MapReduce(mapper=word_count_mapper, reducer=word_count_reducer)

    # Execute MapReduce on the documents.
    result = map_reduce.execute(data=documents, num_workers=4)

    # Display the results
    for word, count in result:
        print(f"{word}: {count}")

hello: 3
world: 2
from: 1
the: 1
other: 1
side: 1
there: 1
of: 1
programming: 1
